In [ ]:
import os
import time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow.keras.backend as K
from tensorflow.keras import activations
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,BatchNormalization,Lambda,Activation,Input,Flatten,Conv2DTranspose,concatenate,Reshape,LeakyReLU

# Set some options and read file names and attribute data

In [ ]:
#options
IMG_DIM = 64
DIM_S = 2
supervised = True #False:Unsupervised
prior_dist = 0 # 0: Isotropic 1: guassian mixture
sensitive = 'emotion' #'gender' or 'emotion'

# automatically set variables based on selected options
DIM_U = 2 if supervised else [IMG_DIM, IMG_DIM, 3]
utility = 'emotion' if sensitive == 'gender' else 'gender'
prior_type = 'Isotropic' if prior_dist == 0 else 'Mixture'
exp_info = f"s_{sensitive}_u_{utility}_{prior_type}_supervised" if supervised else f"s_{sensitive}_{prior_type}_unsupervised"
exp_info += "_P1"

#Folder and image files
generalFolder = './CelebA/'
folder = generalFolder + 'img_align_celeba/'
allFiles = [folder + "/" + f for f in sorted(os.listdir(folder))]#Folder and image files
n = len(allFiles)
print("Number of data : ", n)

df = pd.read_csv(generalFolder + 'list_attr_celeba.csv', nrows=n)
df.head()

# Loading U and S

In [ ]:
S = df['Male'].to_numpy() if(sensitive == 'gender') else df['Smiling'].to_numpy()
S[S == -1] = 0

if supervised:
    U = df['Smiling'].to_numpy() if(sensitive == 'gender') else df['Male'].to_numpy()
    U[U == -1] = 0

# Reading partion information for split data into test, train and validation sets

In [ ]:
eval_part = pd.read_csv(generalFolder + 'list_eval_partition.txt', header=None, sep=" ").to_numpy()[:, 1].astype(int)
print(eval_part.shape)
n_train = sum(eval_part == 0)
n_valid = sum(eval_part == 1)
n_test  = sum(eval_part == 2)
print(n_train, n_valid, n_test)

# Reading image data

# one hot U and S in order to ready them for the model

In [ ]:
x_train = np.zeros((n_train, IMG_DIM, IMG_DIM, 3))
x_valid = np.zeros((n_valid, IMG_DIM, IMG_DIM, 3))
x_test = np.zeros((n_test, IMG_DIM, IMG_DIM, 3))

print("Reading train data:")
for i in range(0,n_train):
  if (i+1) % 20000 == 0:
    print(f"{i+1} / {n_train}")
  img = cv2.imread(allFiles[i])
  img = cv2.resize(img,(IMG_DIM,IMG_DIM),interpolation = cv2.INTER_AREA)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  x_train[i, :, :, :] = img / 255.0

print("Reading validation data:")
for i in range(n_train,(n_train+n_valid)):
  if (i-n_train+1) % 5000 == 0:
    print(f"{i-n_train+1} / {n_valid}")
  img = cv2.imread(allFiles[i])
  img = cv2.resize(img,(IMG_DIM,IMG_DIM),interpolation = cv2.INTER_AREA)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  x_valid[i-n_train, :, :, :] = img / 255.0
    
print("Reading test data:")
for i in range((n_train+n_valid),n):
  if (i-(n_train+n_valid)+1) % 5000 == 0:
    print(f"{i-(n_train+n_valid)+1} / {n_test}")
  img = cv2.imread(allFiles[i])
  img = cv2.resize(img,(IMG_DIM,IMG_DIM),interpolation = cv2.INTER_AREA)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  x_test[i-(n_train+n_valid), :, :, :] = img / 255.0

In [ ]:
def one_hot(labels, num_labels):
    num_labels_data = labels.shape[0]
    one_hot_encoding = np.zeros((num_labels_data,num_labels))
    one_hot_encoding[np.arange(num_labels_data),labels] = 1
    one_hot_encoding = np.reshape(one_hot_encoding, [-1, num_labels])
    return one_hot_encoding

s_train = one_hot(S[:n_train], DIM_S).astype(np.float32)
s_valid = one_hot(S[n_train:(n_train+n_valid)], DIM_S).astype(np.float32)
s_test = one_hot(S[(n_train+n_valid):], DIM_S).astype(np.float32)


if not supervised:
    u_train = x_train
    u_valid = x_valid
    u_test = x_test
else:
    u_train = one_hot(U[:n_train], DIM_U).astype(np.float32)
    u_valid = one_hot(U[n_train:(n_train+n_valid)], DIM_U).astype(np.float32)
    u_test = one_hot(U[(n_train+n_valid):], DIM_U).astype(np.float32)


print(s_train.shape, s_valid.shape, s_test.shape)
print(u_train.shape, u_valid.shape, u_test.shape)
print(DIM_U)

# Define model

In [ ]:
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
def get_encoder(DIM_Z, input_x):
    stride = 2
#     input_x = Input( shape = [IMG_DIM,IMG_DIM,3], name="x" )

    #first hidden layer
    x = Conv2D(16, 3, strides=stride, padding="same", name="enc_h1")(input_x)
    x = BatchNormalization(name="enc_h1_normalized")(x)
    x = Activation(LeakyReLU(), name="enc_h1_activation")(x)
    #second hidden layer
    x = Conv2D(32, 3, strides=stride, padding="same", name="enc_h2")(x)
    x = BatchNormalization(name="enc_h2_normalized")(x)
    x = Activation(LeakyReLU(), name="enc_h2_activation")(x)
    #third hidden layer
    x = Conv2D(64, 3, strides=stride, padding="same", name="enc_h3")(x)
    x = BatchNormalization(name="enc_h3_normalized")(x)
    x = Activation(LeakyReLU(), name="enc_h3_activation")(x)
    #forth hidden layer
    x = Conv2D(128, 3, strides=stride, padding="same", name="enc_h4")(x)
    x = BatchNormalization(name="enc_h4_normalized")(x)
    x = Activation(LeakyReLU(), name="enc_h4_activation")(x)
    #fifth hidden layer
    x = Conv2D(256, 3, strides=stride, padding="same", name="enc_h5")(x)
    x = BatchNormalization(name="enc_h5_normalized")(x)
    x = Activation(LeakyReLU(), name="enc_h5_activation")(x)
    
    shape = K.int_shape(x)
    x = Flatten()(x)
    x = Dense(DIM_Z*4, name="enc_dense_1")(x)
    x = BatchNormalization()(x)
    x = Activation('tanh')(x)

    z_mean = Dense(DIM_Z, name="z_mean")(x)
    z_log_sigma_sq = Dense(DIM_Z, name="z_sigma")(x)
    z = Lambda(sampling, output_shape=DIM_Z, name='lambda_z')([z_mean, z_log_sigma_sq])
    
#     prior_loss = K.mean(-0.5 * K.sum(1 + z_log_sigma_sq - K.square(z_mean) - K.exp(z_log_sigma_sq), axis=-1))
    
    encoder = Model(input_x, z, name = "Encoder")
#     encoder.add_loss((alpha+beta) * prior_loss)
    
    return (encoder, z_mean, z_log_sigma_sq)

In [ ]:
def get_decoder(DIM_Z):
    stride = 2
    input_z = Input(shape = (DIM_Z,), name="z_encoder")
    input_s = Input( shape = (DIM_S,), name="s")

    z_with_s = concatenate([input_z, input_s], name="concat_layer")

    x = Dense(2*2*256, name="dec_dense_1")(z_with_s)
    x = BatchNormalization(name="dec_dense_1_normalized")(x)
    x = Activation('relu')(x)

    #Unflatten
    x = Reshape((2,2,256), name="unflatten1")(x)

    #first hidden layer
    x = Conv2DTranspose(128, 3, strides=stride, padding="same", name="dec_h1")(x)
    x = BatchNormalization(name="dec_h1_normalized")(x)
    x = Activation(LeakyReLU(), name="dec_h1_activation")(x)
    
    #second hidden layer
    x = Conv2DTranspose(64, 3, strides=stride, padding="same", name="dec_h2")(x)
    x = BatchNormalization(name="dec_h2_normalized")(x)
    x = Activation(LeakyReLU(), name="dec_h2_activation")(x)

    #third hidden layer
    x = Conv2DTranspose(32, 3, strides=stride, padding="same", name="dec_h3")(x)
    x = BatchNormalization(name="dec_h3_normalized")(x)
    x = Activation(LeakyReLU(), name="dec_h3_activation")(x)

    #forth hidden layer
    x = Conv2DTranspose(16, 3, strides=stride, padding="same", name="dec_h4")(x)
    x = BatchNormalization(name="dec_h4_normalized")(x)
    x = Activation(LeakyReLU(), name="dec_h4_activation")(x)
    
    #fifth hidden layer
    x = Conv2DTranspose(8, 3, strides=stride, padding="same", name="dec_h5")(x)
    x = BatchNormalization(name="dec_h5_normalized")(x)
    x = Activation(LeakyReLU(), name="dec_h5_activation")(x)
    
    x = Conv2D(3, 3, strides=1, padding="same", name="dec_h6")(x)
    x_hat = Activation("sigmoid", name="x_hat")(x)
    
    decoder = Model([input_z, input_s], x_hat, name = "Uncertainty_Decoder")
    return decoder

In [ ]:
if supervised:
    def get_utility_model(DIM_Z):
        model = Sequential(name="Utility_Decoder_supervised")
        model.add(Dense(DIM_Z, input_dim=DIM_Z))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(DIM_U, activation='softmax', name="u_hat"))
        return model
else:
    def get_utility_model(DIM_Z):
        stride = 2
        model = Sequential(name="Utility_Decoder_unsupervised")
        model.add(Dense(2*2*256, input_dim=DIM_Z))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Reshape((2,2,256)))
        
        model.add(Conv2DTranspose(128, 3, strides=stride, padding="same"))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(64, 3, strides=stride, padding="same"))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(32, 3, strides=stride, padding="same"))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(16, 3, strides=stride, padding="same"))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(8, 3, strides=stride, padding="same"))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2D(3, 3, strides=1, padding="same", activation="sigmoid", name="u_hat"))
        return model

In [ ]:
def get_z_discriminator(DIM_Z):
    model = Sequential(name="Latent_Space_Discriminator")
    
    model.add(Dense(512, input_dim=DIM_Z))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(1, activation="sigmoid"))
    return model

In [ ]:
def get_utility_discriminator():
    model = Sequential(name="Attribute_Class_Discriminator")

    model.add(Dense(DIM_U * 8, input_dim=DIM_U))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(DIM_U * 8,))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(1, activation="sigmoid"))
    return model

In [ ]:
def get_visible_space_discriminator():
    stride = 2
    model = Sequential(name="Visible_Space_Discriminator")
    
    model.add(Input(shape = [IMG_DIM,IMG_DIM,3]))
    
    model.add(Conv2D(16, 3, strides=stride, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(32, 3, strides=stride, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(64, 3, strides=stride, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128, 3, strides=stride, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(1, activation="sigmoid"))
    return model

In [ ]:
def get_prior_generator(DIM_Z, noise_dim=100):
    model = Sequential(name="Prior_Distribution_Generator")
    
    model.add(Dense(noise_dim*2, input_dim=noise_dim))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(noise_dim))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(DIM_Z))
    
    return model

## Build and train a model for evaluating S performance
The following two block cells are **not** part of the framework, it's an evaluator for sensitivity which is required for the experiments

In [ ]:
def get_s_eval_model():
    stride = 2
    model = Sequential(name="CNN_Model_sensitivity_evaluator")
    model.add(Input(shape = [IMG_DIM,IMG_DIM,3]))

    model.add(Conv2D(32, 3, strides=stride, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation(LeakyReLU(alpha=0.2)))

    model.add(Conv2D(64, 3, strides=stride, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation(LeakyReLU(alpha=0.2)))

    model.add(Conv2D(128, 3, strides=stride, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation(LeakyReLU(alpha=0.2)))

    model.add(Conv2D(256, 3, strides=stride, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation(LeakyReLU(alpha=0.2)))

    model.add(Flatten())

    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    model.add(Dense(DIM_S, activation="softmax", name='s_hat'))
    
    return model

In [ ]:
s_eval_model = get_s_eval_model()
s_eval_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='mse')

force_train = False

if not os.path.exists(f"saved_models/celeba_eval_model_s_{sensitive}.h5") or force_train:
    print("Training sensitive attribute evaluator")
    history = s_eval_model.fit(x_train, s_train, validation_data=(x_valid, s_valid), batch_size=2048, epochs=100, shuffle=True, verbose=2)
    s_eval_model.save_weights(f"saved_models/celeba_eval_model_s_{sensitive}.h5")
else:
    print("Loading sensitive attribute evaluator from file")
    s_eval_model.load_weights(f"saved_models/celeba_eval_model_s_{sensitive}.h5")

## Evaluating evaluator!
s_hat_test = s_eval_model.predict(x_test)
print(f"Evaluator accuracy = {(sum(np.argmax(s_hat_test,axis=-1)==np.argmax(s_test,axis=-1)) / s_test.shape[0]) * 100}")

# Putting all togther - define CLUB model

In [ ]:
z_gen_enabled = False #if False, the algorithm sample prior from Normal Distribution

## utility and reconstruction losses
if supervised:
    def loss_u(u_true, u_pred):
        return K.mean(K.sum(K.square(u_true-u_pred), axis=-1))
else:
    def loss_u(u_true, u_pred):
        return K.mean(K.sum(K.square(u_true-u_pred), axis=(1,2,3)))

def loss_x(alpha):
    def loss(inp_x, out_x):
        return alpha * K.mean(K.sum(K.square(inp_x-out_x), axis=(1,2,3)))
    return loss

bce = tf.keras.losses.BinaryCrossentropy()
# Weighted cross-entropy loss
def loss_wce(coef):
    def loss(y, y_pred):
         return coef * bce(y, y_pred)
    return loss

def get_full_model(dim_z, alpha, beta, learning_rate=0.0001, dim_noise = 100):
    ########## Inputs
    input_x = Input( shape=[IMG_DIM,IMG_DIM,3], name="x" )
    input_s = Input( shape = (DIM_S,), name="s" )
    input_z = Input( shape = (dim_z,), name="z" )
    input_noise = Input( shape = (dim_noise,), name="Noise")

    ########## Define AE: Encoder, Utility Decoder and Uncertainty Decoder
    encoder,z_mean,z_log_sigma_sq = get_encoder(dim_z, input_x)
    uncertainty_decoder = get_decoder(dim_z)
    utility_decoder = get_utility_model(dim_z)

    z = encoder(input_x)
    x_hat = uncertainty_decoder([z, input_s])
    u_hat = utility_decoder(z)

    autoencoder = Model([input_x, input_s], [x_hat, u_hat], name="CLUB_Autoencoder")
    prior_loss = (alpha+beta) * K.mean(-0.5 * K.sum(1 + z_log_sigma_sq - K.square(z_mean) - K.exp(z_log_sigma_sq), axis=-1))
    prior_loss = tf.identity(prior_loss, name="kl_loss")
    autoencoder.add_loss(prior_loss)
    autoencoder.compile(loss=[loss_x(alpha), loss_u], optimizer=tf.keras.optimizers.Adam(lr=learning_rate*5))

    ########## Define Latent Space Discriminator
    z_discriminator = get_z_discriminator(dim_z)
    z_discriminator.compile(loss=loss_wce(0.1*(alpha+beta)), optimizer=tf.keras.optimizers.Adam(lr=learning_rate))

    ##########
    #z_discriminator should train separately
    z_discriminator.trainable = False
    prior_generator = get_prior_generator(dim_z, dim_noise)
    prior_gen_zdiscriminator = Model(input_noise, z_discriminator(prior_generator(input_noise)), name="CLUB_generator_zdiscriminator")
    prior_gen_zdiscriminator.compile(loss=loss_wce(-0.1*(alpha+beta)), optimizer=tf.keras.optimizers.Adam(lr=learning_rate))

    ##########
    encoder_zdiscriminator = Model(input_x, z_discriminator(encoder(input_x)), name="CLUB_encoder_zdiscriminator")
    encoder_zdiscriminator.compile(loss=loss_wce(-0.1*(alpha+beta)), optimizer=tf.keras.optimizers.Adam(lr=learning_rate))

    ##########
    u_dircriminator = get_utility_discriminator() if supervised else get_visible_space_discriminator()
    u_dircriminator.compile(loss=loss_wce(0.1), optimizer=tf.keras.optimizers.Adam(lr=learning_rate))

    ##########
    #u_dircriminator should train separately
    u_dircriminator.trainable = False

    if z_gen_enabled:    
        decoder_udiscriminator = Model(input_noise, u_dircriminator(utility_decoder(prior_generator(input_noise))), name="CLUB_decoder_discriminator")
    else:
        decoder_udiscriminator = Model(input_z, u_dircriminator(utility_decoder(input_z)), name="CLUB_decoder_discriminator")

    decoder_udiscriminator.compile(loss=loss_wce(-0.1), optimizer=tf.keras.optimizers.Adam(lr=learning_rate))
    
    return encoder,uncertainty_decoder,utility_decoder,autoencoder,z_discriminator,prior_generator,prior_gen_zdiscriminator,encoder_zdiscriminator,u_dircriminator,decoder_udiscriminator

In [ ]:
# for test
# encoder,uncertainty_decoder,utility_decoder,autoencoder,z_discriminator,prior_generator,prior_gen_zdiscriminator,encoder_zdiscriminator,u_dircriminator,decoder_udiscriminator = get_full_model(32,0.1,0.1)
# tf.keras.utils.plot_model(decoder_udiscriminator, show_shapes=True)
# tf.keras.utils.plot_model(autoencoder, show_shapes=True)

# Pre-training implementation

In [ ]:
def pre_train_ae(dim_z, alpha, beta, force_train=False, max_itr=100, batch_size=1024, verbose=2):
    info_str = f"d_{dim_z}_beta_{beta}_alpha_{alpha}_{exp_info}"
    if not os.path.exists(f"saved_models/celeba_pretrain_{info_str}.h5") or force_train:
        print(f"Pre-Training with {info_str}")
        autoencoder.fit([x_train, s_train], [x_train, u_train], validation_data=([x_valid, s_valid], [x_valid, u_valid]), batch_size=batch_size, epochs=max_itr, shuffle=True, verbose=verbose)
        autoencoder.save_weights(f"saved_models/celeba_pretrain_{info_str}.h5")
    else:
        print(f"Loading model from file with {info_str}")
        autoencoder.load_weights(f"saved_models/celeba_pretrain_{info_str}.h5")

# Full model training loop 

## set prior distribution function based on user config (The following cell will be ignored if the prior-generator network enabled)

In [ ]:
from utils.sampler import gaussian, gaussian_mixture

def sample_prior(latent_dim, batch_size):
    if prior_dist == 0:  
        return gaussian(batch_size, latent_dim)
    elif prior_dist == 1:
        return gaussian_mixture(batch_size, latent_dim, num_labels=min(10, DIM_U))

## customized training loop for block-wised algorithm

In [ ]:
def main_train(dim_z, max_itr=1000, batch_size=1024, z_disc_enabled=True,u_disc_enabled=True,verbose=0, dim_noise=100):
    ones = np.ones((batch_size, 1))
    zeros = np.zeros((batch_size, 1))

    start_time = time.time()
    for epoch in range(max_itr):
        start_time_epoch = time.time()
        # Select a random batch of images
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        x = x_train[idx]
        s = s_train[idx]
        u = u_train[idx]

        # ---------------------
        #  1- Train the Encoder, Utility Decoder, Uncertainty Decoder
        # ---------------------
        ae_loss = autoencoder.train_on_batch([x, s], [x, u])

        if z_disc_enabled:
        # ---------------------
        #  2- Train the Latent Space Discriminator
        # ---------------------

            if z_gen_enabled:
                noise = np.random.uniform(-1.0, 1.0, size=[batch_size, dim_noise])
                latent_prior = prior_generator(noise)
            else:
                latent_prior = sample_prior(dim_z, batch_size)

            idx = np.random.randint(0, x_train.shape[0], batch_size)
            x = x_train[idx]
            latent_enc = encoder(x)

            d_loss_prior = z_discriminator.train_on_batch(latent_prior, zeros)
            d_loss_enc = z_discriminator.train_on_batch(latent_enc, ones)
            dz_loss = d_loss_prior + d_loss_enc

            # ---------------------
            # 3- Train the Encoder and Prior Distribution Generator Adversarially
            # ---------------------
            prior_loss = 0.
            if z_gen_enabled:
                noise = np.random.uniform(-1.0, 1.0, size=[batch_size, dim_noise])
                prior_loss = prior_gen_zdiscriminator.train_on_batch(noise, zeros)

            idx = np.random.randint(0, x_train.shape[0], batch_size)
            x = x_train[idx]
            edz_loss = encoder_zdiscriminator.train_on_batch(x, ones)
        else:
            dz_loss = 0.
            prior_loss = 0.
            edz_loss = 0.

        if u_disc_enabled:
        # ---------------------
        #  4- Train Visible_Space/Attribute_Class Discriminator 
        # ---------------------

            if z_gen_enabled:
                noise = np.random.uniform(-1.0, 1.0, size=[batch_size, dim_noise])
                latent = prior_generator(noise)
            else:
                latent = sample_prior(dim_z, batch_size)
            u_dec = utility_decoder(latent)

            idx = np.random.randint(0, x_train.shape[0], batch_size)
            u = u_train[idx]

            d_loss_real = u_dircriminator.train_on_batch(u, ones)
            d_loss_fake = u_dircriminator.train_on_batch(u_dec, zeros)
            du_loss = d_loss_real + d_loss_fake

        # ---------------------
        #  5- Train the Prior Distribution Generator and Utility Decoder Adversarially
        # ---------------------
            if z_gen_enabled:
                noise = np.random.uniform(-1.0, 1.0, size=[batch_size, dim_noise])
                gdu_loss = decoder_udiscriminator.train_on_batch(noise, zeros)
            else:    
                latent = sample_prior(dim_z, batch_size)
                gdu_loss = decoder_udiscriminator.train_on_batch(latent, zeros)
        else:
            gdu_loss = 0
            du_loss = 0

        # ---------------------
        #  Print stats info
        # ---------------------
        if verbose != 0 and epoch % 50 == 0:
            epoch_time = (time.time() - start_time_epoch)
            print(f"{epoch}, mse:{ae_loss[1]:.4f}, u:{ae_loss[2]:.4f}, dz:{dz_loss:.4f}, edz:{edz_loss:.4f}, prior:{prior_loss:.4f}, du:{du_loss:.4f}, gdu:{gdu_loss:.4f}")
            print(f"Each epoch execution time: {(time.time() - start_time_epoch):.5f} seconds")

    total_time = (time.time() - start_time) / 60
    print(f"Total Execution Time: {total_time:.4f} minutes")

## The following function used for showing the uncertainty-decoder outputs (reconsterction) with different S values on the list of test data

In [ ]:
def save_figures(club, dim_z, alpha, beta):
    info_str = f"d_{dim_z}_beta_{beta}_alpha_{alpha}_{exp_info}"
    
    n_plot_samps = 8
    ind_list = [3, 2, 1, 18, 4 ,15, 11, 0]

    s_hat = s_eval_model.predict(x_test[ind_list])[:, 0]

    rev_s = np.ones((n_plot_samps,2)) - s_test[ind_list]

    x_hat_org, *_ = club.predict([x_test[ind_list], s_test[ind_list] ])
    s_hat = np.append(s_hat, s_eval_model.predict(x_hat_org)[:, 0])

    x_hat_rev, *_ = club.predict([x_test[ind_list], rev_s ])
    s_hat = np.append(s_hat, s_eval_model.predict(x_hat_rev)[:, 0])

    x_hat_no_s, *_ = club.predict([x_test[ind_list], np.zeros((n_plot_samps, DIM_S)) ])
    s_hat = np.append(s_hat, s_eval_model.predict(x_hat_no_s)[:, 0])

    x_hat_all_s, *_ = club.predict([x_test[ind_list], np.ones((n_plot_samps, DIM_S)) ])
    s_hat = np.append(s_hat, s_eval_model.predict(x_hat_all_s)[:, 0])
    
    label = "Male" if sensitive == "gender" else "Smilling"
    s_hat = [f"P({label}) = " + str("{:.2f}".format(1-s)) for s in s_hat]

    from utils.pics_tools import plot_image_grid_with_label
    fig = plot_image_grid_with_label(np.concatenate([x_test[ind_list], x_hat_org, x_hat_rev, x_hat_no_s, x_hat_all_s], axis=0), [IMG_DIM, IMG_DIM, 3], (5,n_plot_samps), s_hat)
    fig.set_size_inches(17, 14)
    plt.subplots_adjust(wspace=0, hspace=0.01)
    plt.savefig(f'./saved_figures_celeba/celeba_{info_str}.svg', format='svg')
    plt.savefig(f'./saved_figures_celeba/celeba_{info_str}.png', format='png')
    plt.show()

# Experiment 1

## Main loop of experiment 1

In [ ]:
from utils.mine import MINE
alpha_list = [0.0001, 0.1, 0.9]
beta_list = [0.0001, 0.1, 0.3, 0.5, 0.8, 0.95]
DIM_Z = [64, 128]

util_acc_tr = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
util_acc_ts = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
util_acc_va = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
sens_mae_tr = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
sens_mae_ts = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
sens_mae_tr_rev = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
sens_mae_ts_rev = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))

sens_acc_tr = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
sens_acc_ts = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
sens_acc_tr_rev = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
sens_acc_ts_rev = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))


mi_u_ts = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
mi_u_tr = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
mi_s_ts = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))
mi_s_tr = np.zeros((len(DIM_Z),len(alpha_list),len(beta_list)))

s_test_rev = np.ones((n_test,2)) - s_test
s_train_rev = np.ones((n_train,2)) - s_train

for i, dimz in enumerate(DIM_Z):
    for j, alpha in enumerate(alpha_list):
        for k, beta in enumerate(beta_list):
            #Pre training
            encoder,uncertainty_decoder,utility_decoder,autoencoder,z_discriminator,prior_generator,prior_gen_zdiscriminator,encoder_zdiscriminator,u_dircriminator,decoder_udiscriminator = get_full_model(dimz, alpha, beta)
            pre_train_ae(dimz, alpha, beta, force_train=False, batch_size=1024, max_itr=50, verbose=0)
            
            print("Full model training")
            main_train(dimz, max_itr=100, batch_size=1024, verbose=1)

            print("Evaluate performance of U")
            z_test = encoder.predict(x_test)
            z_train = encoder.predict(x_train)
            z_valid = encoder.predict(x_valid)
            
            u_test_hat = utility_decoder.predict(z_test)
            u_train_hat = utility_decoder.predict(z_train)
            u_valid_hat = utility_decoder.predict(z_valid)
            if supervised:
                util_acc_ts[i][j][k] = np.mean(np.argmax(u_test_hat,axis=1)==np.argmax(u_test,axis=1)) * 100
                util_acc_tr[i][j][k] = np.mean(np.argmax(u_train_hat,axis=1)==np.argmax(u_train,axis=1)) * 100
                util_acc_va[i][j][k] = np.mean(np.argmax(u_valid_hat,axis=1)==np.argmax(u_valid,axis=1)) * 100
            else:
                util_acc_ts[i][j][k] = np.mean(np.sum(np.square(u_test_hat-u_test), axis=(1,2,3)))
                util_acc_tr[i][j][k] = np.mean(np.sum(np.square(u_train_hat-u_train), axis=(1,2,3)))
                util_acc_va[i][j][k] = np.mean(np.sum(np.square(u_valid_hat-u_valid), axis=(1,2,3)))
            
            save_figures(autoencoder, dimz, alpha, beta)
    
            
            print("Evaluate performance of S")
            # orginal
            x_test_hat = uncertainty_decoder.predict([z_test, s_test])
            s_test_hat = s_eval_model(x_test_hat)
            sens_mae_ts[i][j][k] = np.mean(np.abs(s_test_hat - s_test))
            sens_acc_ts[i][j][k] = np.mean(np.argmax(s_test_hat,axis=1)==np.argmax(s_test,axis=1)) * 100
            
#             x_train_hat = uncertainty_decoder.predict([z_train, s_train])
#             s_train_hat = s_eval_model(x_train_hat)
#             sens_mae_tr[i][j][k] = np.mean(np.abs(s_train_hat - s_train))
#             sens_acc_tr[i][j][k] = np.mean(np.argmax(s_train_hat,axis=1)==np.argmax(s_train,axis=1)) * 100
            
            # reverse
            x_test_hat = uncertainty_decoder.predict([z_test, s_test_rev])
            s_test_hat = s_eval_model(x_test_hat)
            sens_mae_ts_rev[i][j][k] = np.mean(np.abs(s_test_hat - s_test_rev))
            sens_acc_ts_rev[i][j][k] = np.mean(np.argmax(s_test_hat,axis=1)==np.argmax(s_test_rev,axis=1)) * 100
            
#             x_train_hat = uncertainty_decoder.predict([z_train, s_train_rev])
#             s_train_hat = s_eval_model(x_train_hat)
#             sens_mae_tr_rev[i][j][k] = np.mean(np.abs(s_train_hat - s_train_rev))
#             sens_acc_tr_rev[i][j][k] = np.mean(np.argmax(s_train_hat,axis=1)==np.argmax(s_train_rev,axis=1)) * 100
            if supervised:
                print("Evaluate Mutual Information I(Z;U)")            
                mine = MINE(x_dim=dimz, y_dim=DIM_U)
                _, mi_u_ts[i][j][k] = mine.fit(z_test, u_test, epochs=250, batch_size=1024, verbose=0)
                _, mi_u_tr[i][j][k] = mine.fit(z_train, u_train, epochs=250, batch_size=1024, verbose=0)
            
            print("Evaluate Mutual Information I(Z;S)")
            mine = MINE(x_dim=dimz, y_dim=DIM_S)
            _, mi_s_ts[i][j][k] = mine.fit(z_test, s_test, epochs=250, batch_size=1024, verbose=0)
            _, mi_s_tr[i][j][k] = mine.fit(z_train, s_train, epochs=250, batch_size=1024, verbose=0)

# Plot results
## Plot accuracy U

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.rcParams['figure.dpi'] = 300
linestyles = ['-', '--', '-.', ':']
for i in range(util_acc_ts.shape[0]):
    plt.figure(figsize=(15,8))
#     if supervised:
#         plt.title(r'Utility Attribute accuracy ' + f'{exp_info}_d={DIM_Z[i]}', fontsize=16)
#     else:
#         plt.title(r'Utility attribute MSE ' + f'{exp_info}_d={DIM_Z[i]}', fontsize=16)
    for j in range(util_acc_ts.shape[1]):
        if supervised:
            plt.plot(util_acc_ts[i][j], label=r'Utility Att Acc., Test, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0], linewidth=2)
        else:
            plt.plot(util_acc_ts[i][j], label=r'Utility Att MSE., Test, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0], linewidth=2)
            
    plt.legend(prop={'size': 16})
    plt.grid()
    plt.xticks(list(range(len(beta_list))), beta_list, fontsize=18, rotation=90)
    plt.xlabel(r'$\beta$', fontsize=24)
    if supervised:
        plt.ylabel(r'Accuracy on $\mathbf{U}$', fontsize=16)
        plt.savefig(f'./saved_figures/chart_celeba_acc_d_{DIM_Z[i]}_{exp_info}.svg', format='svg', bbox_inches='tight')
        plt.savefig(f'./saved_figures/chart_celeba_acc_d_{DIM_Z[i]}_{exp_info}.png', format='png', bbox_inches='tight')
    else:
        plt.ylabel(r'MSE of $\mathbf{U}$', fontsize=16)
        plt.savefig(f'./saved_figures/chart_celeba_mse_u_d_{DIM_Z[i]}_{exp_info}.svg', format='svg', bbox_inches='tight')
        plt.savefig(f'./saved_figures/chart_celeba_mse_u_d_{DIM_Z[i]}_{exp_info}.png', format='png', bbox_inches='tight')

## Plot accuracy U (for each z and alpha show and save one figure)

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.rcParams['figure.dpi'] = 300
linestyles = ['-', '--', '-.', ':']
for i in range(util_acc_ts.shape[0]):
    for j in range(util_acc_ts.shape[1]):
        plt.figure(figsize=(15,8))
        if supervised:
#             plt.title(r'Utility Attribute accuracy ' + f'{exp_info}_d={DIM_Z[i]}', fontsize=16)
            plt.plot(util_acc_ts[i][j], label=r'Utility Att Acc., Test, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0], linewidth=2)
            plt.main_train(util_acc_tr[i][j], label=r'Utility Att Acc., Train, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[1], linewidth=3)
            plt.plot(util_acc_va[i][j], label=r'Utility Att Acc., Validation, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[3], linewidth=3)
        else:
#             plt.title(r'Utility attribute MSE ' + f'{exp_info}_d={DIM_Z[i]}', fontsize=16)
            plt.plot(util_acc_ts[i][j], label=r'Utility Att MSE., Test, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0], linewidth=2)
            plt.plot(util_acc_tr[i][j], label=r'Utility Att MSE., Train, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[1], linewidth=3)
            plt.plot(util_acc_va[i][j], label=r'Utility Att MSE., Validation, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[3], linewidth=3)
            
        plt.legend(prop={'size': 16})
        plt.grid()
        plt.xticks(list(range(len(beta_list))), beta_list, fontsize=18, rotation=90)
        plt.xlabel(r'$\beta$', fontsize=24)
        if supervised:
            plt.ylabel(r'Accuracy on $\mathbf{U}$', fontsize=16)
            plt.savefig(f'./saved_figures/chart_celeba_acc_d_{DIM_Z[i]}_alpha_{alpha_list[j]}_{exp_info}.svg', format='svg', bbox_inches='tight')
            plt.savefig(f'./saved_figures/chart_celeba_acc_d_{DIM_Z[i]}_alpha_{alpha_list[j]}_{exp_info}.png', format='png', bbox_inches='tight')
        else:
            plt.ylabel(r'MSE of $\mathbf{U}$', fontsize=16)
            plt.savefig(f'./saved_figures/chart_celeba_mse_u_d_{DIM_Z[i]}_alpha_{alpha_list[j]}_{exp_info}.svg', format='svg', bbox_inches='tight')
            plt.savefig(f'./saved_figures/chart_celeba_mse_u_d_{DIM_Z[i]}_alpha_{alpha_list[j]}_{exp_info}.png', format='png', bbox_inches='tight')

## Plot accuracy S

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.rcParams['figure.dpi'] = 300
linestyles = ['-', '--', '-.', ':']
for i in range(sens_acc_ts_rev.shape[0]):
    plt.figure(figsize=(15,8))
#     plt.title(r'Sensitive attribute accuracy ' + f'{exp_info}_d={DIM_Z[i]}', fontsize=16)
    for j in range(sens_acc_ts_rev.shape[1]):
        plt.plot(sens_acc_ts[i][j], label=r'Sensitive Att. Acc., Test, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0], linewidth=2)
        
        plt.plot(sens_acc_ts_rev[i][j], label=r'Sensitive Att. Rev Acc., Test, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0], linewidth=2)

    plt.legend(prop={'size': 16})
    plt.grid()
    plt.xticks(list(range(len(beta_list))), beta_list, fontsize=18, rotation=90)
    plt.xlabel(r'$\beta$', fontsize=24)
    plt.ylabel(r'Accuracy on $\mathbf{S}$', fontsize=16)
    plt.savefig(f'./saved_figures/chart_celeba_acc_s_d_{DIM_Z[i]}_{exp_info}.svg', format='svg', bbox_inches='tight')
    plt.savefig(f'./saved_figures/chart_celeba_acc_s_d_{DIM_Z[i]}_{exp_info}.png', format='png', bbox_inches='tight')

## Plot Mean absolute error (MAE) on all sensitivity states

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.rcParams['figure.dpi'] = 300
linestyles = ['-', '--', '-.', ':']
for i in range(sens_mae_ts.shape[0]):
    plt.figure(figsize=(15,8))
#     plt.title(r'MAE of S ' + f'{exp_info}_d={DIM_Z[i]}', fontsize=16)
    for j in range(sens_mae_ts.shape[1]):
        plt.plot(sens_mae_ts[i][j], label=r'Sensitive Att. MAE., Test, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0], linewidth=2)
        
        plt.plot(sens_mae_ts_rev[i][j], label=r'Sensitivity Rev MAE., Test, $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0], linewidth=2)

    plt.legend(prop={'size': 16})
    plt.grid()
    plt.xticks(list(range(len(beta_list))), beta_list, fontsize=18, rotation=90)
    plt.xlabel(r'$\beta$', fontsize=24)
    plt.ylabel(r'Mean Absolute Error (MAE) on $\mathbf{S}$', fontsize=16)
    plt.savefig(f'./saved_figures/chart_celeba_mae_d_{DIM_Z[i]}_{exp_info}.svg', format='svg', bbox_inches='tight')
    plt.savefig(f'./saved_figures/chart_celeba_mae_d_{DIM_Z[i]}_{exp_info}.png', format='png', bbox_inches='tight')

## Plot mutual information between Z and S

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.rcParams['figure.dpi'] = 300
linestyles = ['-', '--', '-.', ':']
colors = ['b', 'r', 'g', 'c']
for i in range(mi_s_ts.shape[0]):
    plt.figure(figsize=(15,8))
#     plt.title(r'Mutual Information between $\mathbf{S}$ and $\mathbf{Z}$' + f' - dz_{DIM_Z[i]}_{exp_info}', fontsize=20)
    for j in range(mi_s_ts.shape[1]):
        plt.plot(mi_s_ts[i][j], label=r'Test, $d_z=$' + f'{DIM_Z[i]},' + r'$\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0] ,linewidth=3)
#         plt.plot(mi_s_tr[i][j], label=r'Train, $d_z=$' + f'{DIM_Z[i]},' + r'$\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[1], c=colors[j%len(colors)] ,linewidth=3)
        
    # plt.xscale('linear')
    plt.xlabel(r'$\beta$', fontsize=24)
    plt.xticks(list(range(len(beta_list))), beta_list, rotation=90)

    plt.ylabel(r'I($\mathbf{S}$;$\mathbf{Z}$)', fontsize=24)
    plt.legend(prop={'size': 16})
    plt.grid()
    plt.savefig(f'./saved_figures/chart_celeba_mi_zs_dz_{DIM_Z[i]}_{exp_info}.svg', format='svg', bbox_inches='tight')
    plt.savefig(f'./saved_figures/chart_celeba_mi_zs_dz_{DIM_Z[i]}_{exp_info}.png', format='png', bbox_inches='tight')
    plt.show()

## Plot mutual information between Z and U

In [ ]:
if supervised:
    plt.rcParams.update({'font.size': 16})
    plt.rcParams['figure.dpi'] = 300

    for i in range(mi_u_ts.shape[0]):
        plt.figure(figsize=(15,8))
#         plt.title(r'Mutual Information between $\mathbf{U}$ and $\mathbf{Z}$' + f' - {exp_info}_dz_{DIM_Z[i]}', fontsize=20)
        linestyles = ['-', '--', '-.', ':']
        colors = ['b', 'r', 'g', 'c']
        for j in range(mi_u_ts.shape[1]):
            plt.plot(mi_u_ts[i][j], label=r'Test, $d_z=$' + f'{DIM_Z[i]}' + r'$, \alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[0] ,linewidth=3)
    #         plt.plot(mi_u_tr[i][j], label=r'Train, $d_z=$' + f'{DIM_Z[i]}' + r', $\alpha=$' + f"{alpha_list[j]}", linestyle=linestyles[1], c=colors[j%len(colors)] ,linewidth=3)

        # plt.xscale('linear')
        plt.xlabel(r'$\beta$', fontsize=24)
        plt.xticks(list(range(len(beta_list))), beta_list, rotation=90)

        plt.ylabel(r'I($\mathbf{U}$;$\mathbf{Z}$)', fontsize=24)
        plt.legend(prop={'size': 16})
        plt.grid()
        plt.savefig(f'./saved_figures/chart_celeba_mi_zu_d_{DIM_Z[i]}_{exp_info}.svg', format='svg', bbox_inches='tight')
        plt.savefig(f'./saved_figures/chart_celeba_mi_zu_d_{DIM_Z[i]}_{exp_info}.png', format='png', bbox_inches='tight')
        plt.show()

# Save results values to file

In [ ]:
import scipy.io as sio
sio.savemat(f'./saved_data/acc_celeba_{exp_info}.mat', {'util_acc_ts':util_acc_ts, 'util_acc_tr':util_acc_tr, 'util_acc_va':util_acc_va, 'sens_mae_ts':sens_mae_ts, 'sens_mae_tr':sens_mae_tr, 'sens_mae_ts_rev':sens_mae_ts_rev, 'sens_mae_tr_rev':sens_mae_tr_rev,'sens_acc_tr':sens_acc_tr,'sens_acc_ts':sens_acc_ts,'sens_acc_tr_rev':sens_acc_tr_rev,'sens_acc_ts_rev':sens_acc_ts_rev})
sio.savemat(f'./saved_data/mi_celeba_{exp_info}.mat', {'mi_s_ts':mi_s_ts, 'mi_s_tr':mi_s_tr, 'mi_u_ts':mi_u_ts, 'mi_u_tr':mi_u_tr})

# Load results from file

In [ ]:
import scipy.io as sio
mat_contents  = sio.loadmat(f'./saved_data/acc_celeba_{exp_info}.mat')
util_acc_ts = mat_contents['util_acc_ts']
util_acc_tr = mat_contents['util_acc_tr']
util_acc_va = mat_contents['util_acc_va']
sens_acc_tr = mat_contents['sens_acc_tr']
sens_acc_ts = mat_contents['sens_acc_ts']
sens_acc_tr_rev = mat_contents['sens_acc_tr_rev']
sens_acc_ts_rev = mat_contents['sens_acc_ts_rev']
sens_mae_ts = mat_contents['sens_mae_ts']
sens_mae_tr = mat_contents['sens_mae_tr']
sens_mae_ts_rev = mat_contents['sens_mae_ts_rev']
sens_mae_tr_rev = mat_contents['sens_mae_tr_rev']


mat_contents  = sio.loadmat(f'./saved_data/mi_celeba_{exp_info}.mat')
mi_s_tr = mat_contents['mi_s_tr']
mi_s_ts = mat_contents['mi_s_ts']
mi_u_ts = mat_contents['mi_u_ts']
mi_u_tr = mat_contents['mi_u_tr']

In [ ]:
alpha_list = [0.0001, 0.1, 0.9]
beta_list = [0.0001, 0.1, 0.3, 0.5, 0.8, 0.95]
DIM_Z = [64, 128]